In [3]:
# VENV name: cudaconfig
import torch

print("CUDA is available: ", torch.cuda.is_available())
# True

print("Number of CUDA devices: ", torch.cuda.device_count())
# 5

print("CUDA current device: ", torch.cuda.current_device())
# 0

print("CUDA device name: ", torch.cuda.get_device_name(0))
# 'NVIDIA GeForce RTX 3060's

CUDA is available:  False
Number of CUDA devices:  0


AssertionError: Torch not compiled with CUDA enabled

In [2]:
import torch
x = torch.rand(5, 3)
print(x)

tensor([[0.9228, 0.8818, 0.9864],
        [0.3102, 0.7416, 0.7359],
        [0.5965, 0.7917, 0.7936],
        [0.1401, 0.8968, 0.8016],
        [0.2034, 0.8978, 0.3374]])


In [6]:
from fastai.vision.all import *
import pandas as pd

# Load the labels CSV
labels_df = pd.read_csv('data/test/controls.csv')

# Path to images
path = Path('data/test')

# Define a function to get the control inputs for each image
def get_y(row):
    return tensor([row['steering'], row['throttle'], row['brake']])

def get_x(row):
    return path / row['frame']  # Using '/' for path concatenation

# DataBlock for regression task
dblock = DataBlock(
    blocks=(ImageBlock, RegressionBlock),  # Images as input, continuous labels as output
    get_x=get_x,  # Get image path
    get_y=get_y,  # Get control inputs as a tensor
    splitter=RandomSplitter(valid_pct=0.2),  # Split train/validation
    item_tfms=Resize((398, 94)),  # Resize images
    batch_tfms=aug_transforms()  # Apply augmentations during training
)

# Create DataLoaders
dls = dblock.dataloaders(labels_df)

# Define the learner with a pretrained ResNet model
learn = vision_learner(
    dls, 
    resnet34, 
    loss_func=MSELossFlat(),
    y_range=(-1., 1.)  # Example y_range for normalized outputs; adjust as needed for your task
)

print("Training the model")
# Train the model for a few epochs
learn.fine_tune(10)

# Evaluate the model on the validation set
print("Evaluating the model")
learn.validate()

# Save the entire model for future use
learn.export('trackmania_learner.pkl')

Training the model


epoch,train_loss,valid_loss,time
0,0.728773,0.585586,00:45


epoch,train_loss,valid_loss,time
0,0.638688,0.455145,01:01
1,0.628212,0.424364,01:06
2,0.604955,0.305409,01:06
3,0.585382,0.444560,01:06
4,0.547881,0.322901,01:02
5,0.516271,0.280230,01:00
6,0.485756,0.270969,01:00
7,0.455555,0.251855,01:00
8,0.432079,0.247618,01:01
9,0.415165,0.234608,01:00


Evaluating the model
